In [1]:
print("hello")

hello


In [2]:
pip install -U langchain langchain-aws boto3

  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 5.7 MB/s eta 0:00:00a 0:00:01
Using cached pydantic_core-2.33.2-cp313-cp313-macosx_11_0_arm64.whl (1.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.4 MB/s eta 0:00:00a 0:00:01
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached typing_inspection-0.4.1-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.4/633.4 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update,

In [1]:
import boto3
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage

# 1. Instantiate the ChatBedrock class
# By default, it uses the credentials and region from your environment.
llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs={"temperature": 0.7},
)

In [4]:
messages = [
    HumanMessage(
        content="Hello! Can you explain what a Large Language Model is in one sentence?"
    )
]

# 3. Invoke the model
response = llm.invoke(messages)

# 4. Print the response content
print(response.content)

A Large Language Model is an artificial intelligence system trained on vast amounts of text data to understand and generate human-like language for various natural language processing tasks.


In [7]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define the model
llm = ChatBedrock(
    model_id="amazon.titan-text-premier-v1:0",
    model_kwargs={"temperature": 0.5}
)

# Define a prompt template
prompt_template = ChatPromptTemplate.from_template(
    "You are a helpful assistant. Answer the following question clearly and concisely.\nQuestion: {question}"
)

# Define an output parser to get the string content from the response
output_parser = StrOutputParser()

# Create the chain using the | (pipe) operator from LCEL
chain = prompt_template | llm | output_parser

# Run the chain
question = "What is the primary benefit of using AWS Bedrock?"
response = chain.invoke({"question": question})

print(response)

AWS Bedrock offers several benefits, including:

1. **Ease of use**: AWS Bedrock simplifies the process of building and deploying LLM-powered applications by providing a unified API and pre-configured models.
2. **Cost savings**: AWS Bedrock offers flexible pricing options, allowing customers to pay only for the compute resources they use.
3. **Improved performance**: AWS Bedrock is built on AWS's scalable infrastructure, providing fast and reliable access to LLMs.
4. **Security and compliance**: AWS Bedrock integrates with AWS's security and compliance features, providing customers with peace of mind when using LLMs.
5. **Access to cutting-edge LLMs**: AWS Bedrock provides access to a range of LLMs, including the latest models from leading AI research organizations.

Overall, AWS Bedrock provides a convenient and cost-effective way for developers to build and deploy LLM-powered applications, with access to cutting-edge models and AWS's scalable infrastructure.


In [8]:
pip install pypdf faiss-cpu 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 4.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langchain-community

  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 6.7 MB/s eta 0:00:00a 0:00:01
Using cached mypy_extensions-1.1.0-py3-none-any.whl (5.0 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_aws import ChatBedrock, BedrockEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# 1. Initialize Bedrock LLM and Embeddings model
# Note: We use a separate model for creating embeddings (numerical representations of text).
llm = ChatBedrock(model_id="anthropic.claude-3-haiku-20240307-v1:0")
embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1")

In [5]:
# 2. Load and process your document
loader = PyPDFLoader("~/Desktop/Personal/Docs/10958-Suraj-Salunke-Resume.pdf")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 3. Create a vector store
# This creates vector embeddings and stores them for efficient searching.
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()

# 4. Create a prompt template for the RAG chain
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# 5. Create the RAG chain
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# 6. Invoke the chain with a question
response = retrieval_chain.invoke({"input": "What was the main finding mentioned in the document?"})

print(response["answer"])

Based on the provided context, the main finding mentioned in the document is that the candidate was involved in several projects and initiatives that led to significant improvements in various areas:

1. Migrated applications from EC2 to EKS and ECS to EC2, optimizing infrastructure and achieving cost savings of ~$1500/month.

2. Streamlined system architecture by merging two closely related bounded contexts into one, reducing database and Kubernetes resource usage by ~50%, leading to significant cost savings and operational efficiency.

3. Resolved long-standing customer escalations in legacy paths and channels curation, improving system stability and enhancing customer satisfaction.

4. Migrated legacy assessment data to new Kafka event stream, rewriting the backend from Python to Node.js, and redesigning the UI in React using new Figma designs for improved usability.

5. Delivered a content tool for assessment questions, automating CSV data processing and integration with assessment

In [7]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field
import random

# --- Tool 1: Get Weather (using @tool decorator for a simple case) ---
@tool
def get_current_weather(location: str) -> str:
    """Gets the current weather for a specified location."""
    if "mumbai" in location.lower():
        return f"The weather in {location} is 32°C and sunny."
    elif "delhi" in location.lower():
        return f"The weather in {location} is 35°C with clear skies."
    else:
        return f"The weather in {location} is {random.randint(25, 35)}°C."

# --- Tool 2: Get Stock Price (using Pydantic for a more structured schema) ---
class StockPriceInput(BaseModel):
    ticker_symbol: str = Field(description="The stock ticker symbol, e.g., 'AMZN' for Amazon.")

@tool(args_schema=StockPriceInput)
def get_stock_price(ticker_symbol: str) -> float:
    """Gets the current stock price for a given ticker symbol."""
    # In a real app, you would call an API here.
    if ticker_symbol.upper() == "AMZN":
        return 185.57
    elif ticker_symbol.upper() == "GOOGL":
        return 179.63
    else:
        return random.uniform(100.0, 500.0)

# Create a list of the tools you want the model to have access to
tools = [get_current_weather, get_stock_price]

In [8]:
from langchain_aws import ChatBedrock

# Choose a model that supports tool calling, like Claude 3 Sonnet or Haiku
# Haiku is often a great choice for this as it's fast and cost-effective.
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

# Instantiate the model
model = ChatBedrock(
    model_id=model_id,
    model_kwargs={"temperature": 0.0}
)

# Bind the tools to the model
# This creates a new model instance that knows about the tools.
model_with_tools = model.bind_tools(tools)

In [9]:
from langchain_core.messages import HumanMessage

# --- Example 1: A query that should trigger the weather tool ---
query1 = "What's the weather like in Mumbai?"
print(f"User Query: {query1}\n")

# The first response from the model will NOT be a final answer.
# It will be an instruction to call a tool.
ai_response = model_with_tools.invoke([HumanMessage(query1)])

print("--- Initial Model Response ---")
print(ai_response)
print("-" * 20)
# This will print an AIMessage containing tool_calls, like:
# AIMessage(
#   content='',
#   additional_kwargs={...},
#   response_metadata={...},
#   tool_calls=[{
#       'name': 'get_current_weather',
#       'args': {'location': 'Mumbai'},
#       'id': 'tool_call_abc123'
#   }]
# )

# The AIMessage object has a .tool_calls attribute which is a list of parsed tool calls
print(f"Tool Calls: {ai_response.tool_calls}\n")

User Query: What's the weather like in Mumbai?

--- Initial Model Response ---
content='' additional_kwargs={'usage': {'prompt_tokens': 424, 'completion_tokens': 55, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 479}, 'stop_reason': 'tool_use', 'thinking': {}, 'model_id': 'anthropic.claude-3-haiku-20240307-v1:0', 'model_name': 'anthropic.claude-3-haiku-20240307-v1:0'} response_metadata={'usage': {'prompt_tokens': 424, 'completion_tokens': 55, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 479}, 'stop_reason': 'tool_use', 'thinking': {}, 'model_id': 'anthropic.claude-3-haiku-20240307-v1:0', 'model_name': 'anthropic.claude-3-haiku-20240307-v1:0'} id='run--cc903bbf-4e40-4303-b3d0-1ae91e1b787a-0' tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'Mumbai'}, 'id': 'toolu_bdrk_01Lqe5zuBodMEQhdnUrvDoTH', 'type': 'tool_call'}] usage_metadata={'input_tokens': 424, 'output_tokens': 55, 'total_tokens': 479, 'input_token_de

In [10]:
from langchain_core.messages import ToolMessage

# Now we need to manually execute the tool and return the result
# In a real app, this would be a loop over all tool_calls.
tool_call = ai_response.tool_calls[0]
tool_name = tool_call['name']
tool_args = tool_call['args']
tool_call_id = tool_call['id']

# Find and run the actual Python function
if tool_name == "get_current_weather":
    tool_output = get_current_weather.invoke(tool_args)
elif tool_name == "get_stock_price":
    tool_output = get_stock_price.invoke(tool_args)

print("--- Executing Tool ---")
print(f"Running: {tool_name}({tool_args})")
print(f"Output: {tool_output}\n")
# Output:
# Running: get_current_weather({'location': 'Mumbai'})
# Output: The weather in Mumbai is 32°C and sunny.

# --- Now, call the model AGAIN with the tool's output ---
# We append the original AI message and the new ToolMessage to the history
messages = [
    HumanMessage(query1),
    ai_response, # The message containing the tool call instruction
    ToolMessage(content=str(tool_output), tool_call_id=tool_call_id)
]

final_response = model_with_tools.invoke(messages)

print("--- Final Model Response ---")
print(final_response.content)

--- Executing Tool ---
Running: get_current_weather({'location': 'Mumbai'})
Output: The weather in Mumbai is 32°C and sunny.

--- Final Model Response ---
The current weather in Mumbai is 32°C and sunny.
